In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Import data
df = pd.read_csv('datasets/ealing.csv', parse_dates=True)
df.head()

In [ ]:
# Use Ealing as an example to determine model initial conditions
infected_ts = df['infected'].tolist()

P = 342000 # Ealing population ONS 2018 mid year
I = 8      # Infected people at 14/03/2020
R = 0      # Recovered people at 29/03/2020
n_days = len(df)

# Input parameters
beta = 0.38 # Per day
alpha = 2.5 * beta # WHO estimate

# Calculate initial conditions in terms of total number of individuals
S0 = (P - I)
I0 = I
R0 = R    # Recovered people

# Construct vector of parameters
params = [alpha, beta]

# Construct vector of initial conditions
w0 = [S0, I0, R0]

In [ ]:
# Import model
from opensir.models import SIR, SIRX

# Initialize an emplty SIR model
my_SIR = SIR()

# Set model parameters, solve and unpack solution
my_SIR.set_params(p=params, initial_conds=w0)
my_SIR.solve(tf_days=n_days-1, numpoints=n_days)
sol = my_SIR.fetch()

# Unpack the numerical solution for the susceptible (S), infected (I) and recovered or removed (R)
S_sir = sol[:,1]
I_sir = sol[:,2]
R_sir = sol[:,3]

In [ ]:
# Plot the results

days_list = df['date'].tolist()

fig = go.Figure()
fig.update_layout(title='Initial model vs. data')
fig.add_trace(go.Scatter(x=days_list, y=I_sir, mode='lines+markers', name='SIR_model'))
fig.add_trace(go.Scatter(x=days_list, y=infected_ts, mode='markers', name='data'))
fig.show()

In [ ]:
# Create SIR with default parameters
my_SIR_fitted = SIR()
my_SIR_fitted.set_params(params,w0)

# Fit parameters
w = my_SIR_fitted.fit(df.index.tolist(), infected_ts, P, fit_index=[True,False])
# Print the fitted reproduction rate
print("Fitted reproduction rate R_0 = %.2f" % my_SIR_fitted.r0)

# Build the new solution
my_SIR_fitted.solve(tf_days=n_days-1, numpoints=n_days)
# Extract solution
sol = my_SIR_fitted.fetch()

I_sir_fitted = sol[:,2]

# Plot the results
fig = go.Figure()
fig.update_layout(title='Fitted model vs. data')
fig.add_trace(go.Scatter(x=days_list, y=I_sir_fitted, mode='lines+markers', name='SIR_model'))
fig.add_trace(go.Scatter(x=days_list, y=infected_ts, mode='markers', name='data'))
fig.show()

# Examples

In [ ]:
# Long term simulation - DANGER: the longer the timeframe, the less accurate the model becomes.
long_term_days = 90
# Convert into seconds
tf_long = long_term_days-1
sol_long = my_SIR_fitted.solve(tf_long, long_term_days).fetch()
N_S_long = sol_long[:,1]
N_I_long = sol_long[:,2]
N_R_long = sol_long[:,3]

tspan_long = np.linspace(0,tf_long,long_term_days)

# Plot the number of susceptible, infected and recovered in a two months period

S_trace = go.Scatter(x=tspan_long, y=N_S_long, mode='lines', name='Susceptible')
I_trace = go.Scatter(x=tspan_long, y=N_I_long, mode='lines', name='Infected')
R_trace = go.Scatter(x=tspan_long, y=N_R_long, mode='lines', name='Recovered or Removed')

'''
# Using subplots
fig = make_subplots(rows=1, cols=3)
fig.update_layout(title='Long term model')
fig.add_trace(S_trace, row=1, col=1)
fig.add_trace(I_trace, row=1, col=2)
fig.add_trace(R_trace, row=1, col=3)
fig.show()
'''

# All in one
fig = go.Figure()
fig.update_layout(title='Long term model')
fig.add_trace(S_trace)
fig.add_trace(I_trace)
fig.add_trace(R_trace)
fig.show()

In [ ]:
# Sensitivity analysis
def compare_infections(model, tf, numpoints, alpha_list=2.5, abserr=1e-8, relerr=1e-6):
    """ compare_infections compare SIR model predictions against
    a list of alpha values
    
    Inputs:
    w0: Initial conditions
    t: Time vector /
    alpha_list: list or numpy array of values of alpha to be tested
    
    Outputs:
    S_list: List of predictions for the fraction of susceptible population for each alpha
    I_list: List of predictions for the fraction of infected population for each alpha
    R_list: List of predictions for the fraction of recovered/removed population for each alpha
    """
    S_list, I_list, R_list = [], [], []
    
    for i in alpha_list:
        # Update parameter list
        model.p[0] = i
        wsol=model.solve(tf,numpoints).fetch()
        S_list.append(wsol[:,1])
        I_list.append(wsol[:,2])
        R_list.append(wsol[:,3]) 
    return S_list, I_list, R_list

In [ ]:
alpha_list = beta*np.array([1.5,1.6,1.7])
S_list, I_list, R_list = compare_infections(my_SIR, tf_long, long_term_days, alpha_list)

In [ ]:
n_sens = len(S_list)


fig = make_subplots(rows=1, cols=3)
fig.update_layout(title='Sensitivity analysis on Reproduction Rate R0')

R0_list = [1.5, 1.6, 1.7]
color_list = ['blue','black','red']
dash_list = ['dot', 'solid', 'dashdot']

for i in range(n_sens):
    if True:
        R0 = R0_list[i]
        line_properties = dict(color=color_list[i], width=1.5, dash=dash_list[i])
        S_trace = go.Scatter(x=tspan_long, y=S_list[i], mode='lines', name='Susceptible - %s' % R0, 
                             line=line_properties)
        I_trace = go.Scatter(x=tspan_long, y=I_list[i], mode='lines', name='Infected - %s' % R0, 
                             line=line_properties)
        R_trace = go.Scatter(x=tspan_long, y=R_list[i], mode='lines', name='Recovered or Removed - %s' % R0, 
                             line=line_properties)

        fig.add_trace(S_trace, row=1, col=1)
        fig.add_trace(I_trace, row=1, col=2)
        fig.add_trace(R_trace, row=1, col=3)

fig.show()

# TODO:  fix legend